<a href="https://colab.research.google.com/github/coderTanisha22/Jarvis_learnings/blob/main/hugging_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers #pretrained AI models access
!pip install torch #pytorch backened use to run models

In [ ]:
!pip install gradio transformers torch #for maing a new web interface for chatbot

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
chatbot_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
chathis_tokens = None

In [4]:
def generate_response(messages, chatbot=None):
    global chathis_tokens
    try:
        # If messages is a string (fallback), wrap it in dict format
        if isinstance(messages, str):
            messages = [{"role": "user", "content": messages}]

        # Get the latest user message
        user_message = messages[-1]["content"]

        # Tokenize and prepare input
        user_tokens = tokenizer.encode(user_message + tokenizer.eos_token, return_tensors='pt')
        model_input = torch.cat([chathis_tokens, user_tokens], dim=-1) if chathis_tokens is not None else user_tokens

        # Generate response
        response_tokens = chatbot_model.generate(
            model_input,
            max_length=1000,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

        # Decode response and append to messages
        new_tokens = response_tokens[0][model_input.shape[-1]:]
        bot_reply = tokenizer.decode(new_tokens, skip_special_tokens=True)
        messages.append({"role": "assistant", "content": bot_reply})
        #update chat history for next turn
        chathis_tokens = response_tokens

        return messages

    except Exception as e:
        # Always return messages as a list
        if not isinstance(messages, list):
            messages = [{"role": "user", "content": str(messages)}]
        messages.append({"role": "assistant", "content": f"[Error] {str(e)}"})
        return messages

In [5]:
import gradio as gr
gr.ChatInterface(fn=generate_response, type="messages").launch(debug=True) #so that it shows the error messege; it's not hidden

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cb81e9a861e72dee1d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cb81e9a861e72dee1d.gradio.live
